# Финальное задание

In [1]:
import pandas as pd
import numpy as np
import time
import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

#### Пример чтения файла с признаками

In [2]:
data = pd.read_csv('./features.csv', index_col='match_id')
data.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


## Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках?

In [3]:
data.isnull().any().any()

True

In [4]:
passes = data.count()[data.count() != data.shape[0]]
print(f'Столбцы, где есть пропуски \n{passes}')

Столбцы, где есть пропуски 
first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64


Признаки события "первая кровь" (first blood). Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение

Заменим пропуски нулями

In [5]:
data.fillna(0, inplace = True)
data.isnull().any().any()

False

## Как называется столбец, содержащий целевую переменную?

Выделим целевую переменную

In [6]:
y = data['radiant_win'] # целевая переменная
X = data.drop(columns='radiant_win')

Даанные поля отсутствуют в тестовой выборке
* duration: длительность
* tower_status_radiant
* tower_status_dire
* barracks_status_radiant
* barracks_status_dire

In [7]:
X.drop(['duration',
        'tower_status_radiant',
        'tower_status_dire', 
        'barracks_status_radiant',
        'barracks_status_dire'], axis='columns', inplace=True)

## Подход 1: градиентный бустинг "в лоб"

In [9]:
kf = KFold(n_splits=5, shuffle=True, random_state=7979)
result = []
n_estimators = [10, 20, 30, 50, 75]
for n in n_estimators:
    print(f'n_estimators={n}')
    model = GradientBoostingClassifier(n_estimators=n, random_state=7979)
    start_time = datetime.datetime.now()
    cross_model = cross_val_score(model, X, y, cv=kf, scoring='roc_auc', n_jobs=-1).mean()
    print('time:', datetime.datetime.now() - start_time)
    print(f'score:{cross_model}')
    print()
    result.append(cross_model)

n_estimators=10
time: 0:00:52.226770
score:0.664989762526323

n_estimators=20
time: 0:01:36.004326
score:0.6823347677461424

n_estimators=30
time: 0:02:23.437137
score:0.6894658389438937

n_estimators=50
time: 0:03:44.994584
score:0.6977346926873833

n_estimators=75
time: 0:05:10.498963
score:0.7034355902749001



## Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Какое качество при этом получилось? 

n_estimators=30  
Time: 0:02:19.958300  
score:0.6894658389438937

## Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? 
Имеет  
n_estimators=100  
Time: 0:08:41.009447  
score:0.7066524497035768

## Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?
 Изменение параметров : увеличение learning_rate и  уменьшение max_depth, использовать для обучения и кросс-валидации не всю выборку, а некоторое ее подмножество

## Подход 2: логистическая регрессия

## Оцените качество логистической регрессии с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга.

In [10]:
scale = StandardScaler()
X_scale = scale.fit_transform(X)

In [11]:
kf = KFold(n_splits=5, shuffle=True, random_state=7979)
result = []
C_params = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
for c in C_params:
    print(f'C={c}')
    model = LogisticRegression(C=c, random_state=42, n_jobs=-1)
    start_time = datetime.datetime.now()
    cross_model = round(cross_val_score(model, X_scale, y, cv=kf, scoring='roc_auc', n_jobs=-1).mean(),4)
    print('time:', datetime.datetime.now() - start_time)
    print(f'score:{cross_model}')
    print()
    result.append(cross_model)

C=0.0001
time: 0:00:09.282298
score:0.7112

C=0.001
time: 0:00:04.305857
score:0.7162

C=0.01
time: 0:00:05.092629
score:0.7164

C=0.1
time: 0:00:05.265701
score:0.7163

C=1
time: 0:00:05.001862
score:0.7163

C=10
time: 0:00:05.109512
score:0.7163

C=100
time: 0:00:04.775612
score:0.7163

C=1000
time: 0:00:04.978669
score:0.7163

C=10000
time: 0:00:05.383874
score:0.7163



##  Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось?
C=0.01  
time: 0:00:04.531124  
score:0.7164

## Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

Качество выше. Логистическая регрессия работает заметно быстрее. Для полноценного сравнения моделей требуется более полный подбор параметров, особенно для бустинга

## Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [12]:
X_del_categ = X.drop(columns=['lobby_type',
                               'r1_hero','r2_hero','r3_hero','r4_hero','r5_hero',
                              'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero'])
scale = StandardScaler()
X_scale_del_categ = scale.fit_transform(X_del_categ)

In [13]:
kf = KFold(n_splits=5, shuffle=True, random_state=7979)
result = []
C_params = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
for c in C_params:
    print(f'C={c}')
    model = LogisticRegression(C=c, random_state=42, n_jobs=-1)
    start_time = datetime.datetime.now()
    cross_model = round(cross_val_score(model, X_scale_del_categ, y, cv=kf, scoring='roc_auc', n_jobs=-1).mean(),4)
    print('time:', datetime.datetime.now() - start_time)
    print(f'score:{cross_model}')
    print()
    result.append(cross_model)

C=0.0001
time: 0:00:02.388437
score:0.7112

C=0.001
time: 0:00:03.378058
score:0.7162

C=0.01
time: 0:00:04.498125
score:0.7163

C=0.1
time: 0:00:05.560032
score:0.7163

C=1
time: 0:00:04.362172
score:0.7163

C=10
time: 0:00:04.392198
score:0.7163

C=100
time: 0:00:04.624873
score:0.7163

C=1000
time: 0:00:04.905507
score:0.7163

C=10000
time: 0:00:04.532049
score:0.7163



Отличия нет. Вероятно модель давала этим признакам веса близкие к нулю

## На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [14]:
X['r1_hero'].nunique(), X['d1_hero'].nunique()

(108, 108)

## Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [15]:
heroes = [i for i in X.columns if 'hero' in i]
heroes_uniq = np.unique(X[heroes])

In [16]:
new_columns = {}
for i in heroes_uniq:
    r = [(data['r%d_hero' % n] == i) + 0 for n in range(1, 6)]
    d = [(data['d%d_hero' % n] == i) + 0 for n in range(1, 6)]
    new_columns['hero%s' % i] = sum(r) - sum(d)
X_new = data.assign(**new_columns)
X_new.sort_index(inplace=True)

In [17]:
y = X_new ['radiant_win']
X_new.drop(['duration', 
            'radiant_win', 
            'tower_status_radiant', 'tower_status_dire',
            'barracks_status_radiant', 'barracks_status_dire'], axis='columns', inplace=True)

In [18]:
scaler = StandardScaler()
X_scale_new = scaler.fit_transform(X_new)

## Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [19]:
result = []
C_params = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
for c in C_params:
    print(f'C={c}')
    model = LogisticRegression(C=c, random_state=7979, n_jobs=-1)
    start_time = datetime.datetime.now()
    cross_model = round(cross_val_score(model, X_scale_new, y, cv=kf, scoring='roc_auc', n_jobs=-1).mean(),4)
    print('time:', datetime.datetime.now() - start_time)
    print(f'score:{cross_model}')
    print()
    result.append(cross_model)

C=0.0001
time: 0:00:04.634346
score:0.7427

C=0.001
time: 0:00:06.808080
score:0.7515

C=0.01
time: 0:00:09.030301
score:0.7518

C=0.1
time: 0:00:10.408888
score:0.7518

C=1
time: 0:00:10.050641
score:0.7518

C=10
time: 0:00:10.716363
score:0.7518

C=100
time: 0:00:10.018021
score:0.7518

C=1000
time: 0:00:10.273381
score:0.7518



Лучшее качество  С = 0.01 (Значение 0.7518). Информация о героях является важным признаком, который необходимо правильно обработать

## Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей

In [20]:
test = pd.read_csv('features_test.csv', index_col='match_id')

In [21]:
test.fillna(0, inplace = True)
test.isnull().any().any()

False

In [22]:
heroes = [i for i in test.columns if 'hero' in i]
heroes_uniq = np.unique(test[heroes])
len(heroes_uniq)

108

In [23]:
new_columns_test = {}
for i in heroes_uniq:
    r = [(test['r%d_hero' % n] == i) + 0 for n in range(1, 6)]
    d = [(test['d%d_hero' % n] == i) + 0 for n in range(1, 6)]
    new_columns['hero%s' % i] = sum(r) - sum(d)
X_test = test.assign(**new_columns)
X_test.sort_index(inplace=True)

In [24]:
X_scaler_test = scaler.transform(X_test)

In [25]:
lr_model = LogisticRegression(C=0.01).fit(X_scale_new, y)

predicted = lr_model.predict_proba(X_scaler_test)
pred_win_R = []
for i in predicted:
    pred_win_R.append(i[1])

## Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

In [26]:
print('Минимальное значение:', min(pred_win_R))
print('Максимальное значение:', max(pred_win_R))

Минимальное значение: 0.008707836522637425
Максимальное значение: 0.9963101334700569
